
# Random Search


Installation



In [ ]:
# To install the required package, use the following command:
# !pip install modeva

Authentication



In [ ]:
# To get authentication, use the following command: (To get full access please replace the token to your own token)
# from modeva.utils.authenticate import authenticate
# authenticate(token='eaaa4301-b140-484c-8e93-f9f633c8bacb')

Import required modules



In [ ]:
from modeva import DataSet
from modeva import TestSuite
from modeva.models import MoElasticNet
from modeva.models import ModelTuneRandomSearch

Load Dataset



In [ ]:
ds = DataSet()
ds.load(name="BikeSharing")
ds.set_random_split()

ds.scale_numerical(features=("cnt",), method="log1p")
ds.preprocess()

## Run random search



In [ ]:
param_grid = {"alpha": [0.1, 1.0, 10],
              "l1_ratio": [(i + 1) * 0.1 for i in range(10)]}

model = MoElasticNet()
hpo = ModelTuneRandomSearch(dataset=ds, model=model)
result = hpo.run(param_distributions=param_grid,
                 n_iter=20,
                 metric="MSE",
                 cv=5)
result.table

In [ ]:
result.plot("parallel", figsize=(8, 6))

In [ ]:
result.plot(("alpha", "MSE"))

In [ ]:
result.plot(("l1_ratio", "MSE"))

## Retrain model with best hyperparameter



In [ ]:
model_tuned = MoElasticNet(**result.value["params"][0],
                           name="GLM-Tuned")
model_tuned.fit(ds.train_x, ds.train_y)
model_tuned

## Diagnose the tuned model



In [ ]:
ts = TestSuite(ds, model_tuned)
result = ts.diagnose_accuracy_table()
result.table